# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
WeatherPy_df = pd.read_csv("../WeatherPy/Output/WeatherPy_Output.csv")
WeatherPy_df


,City Name,City ID,Country Code,Longitude,Latitude,Temp,Feels Like Temp,Min Temp,Max Temp,Pressure,Humidity,Visibility,Wind Speed,Wind Temp,Sunrise,Sunset,Cloudiness
0,Barrow,5880054,US,-156.7887,71.2906,-26.99,-33.99,-26.99,-26.99,1018,69,10000,4.12,240,1647707355,1647751226,100
1,Castro,3466704,BR,-50.0119,-24.7911,18.37,18.85,18.37,18.37,1017,99,8193,1.10,326,1647681792,1647725547,100
2,Abadan,162099,TM,58.2056,38.0507,7.85,4.45,7.85,7.85,1026,71,10000,6.17,110,1647655970,1647699436,100
3,Yellowknife,6185377,CA,-114.3525,62.4560,-21.38,-28.05,-21.38,-21.38,1014,100,3683,2.34,76,1647697284,1647740932,95
4,Upernavik,3418910,GL,-56.1549,72.7868,-22.28,-27.36,-22.28,-22.28,1012,70,10000,1.57,28,1647683268,1647727019,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,Merauke,2082539,ID,140.3333,-8.4667,27.26,30.55,27.26,27.26,1007,82,10000,1.59,255,1647636169,1647679824,96
539,Chaman,1181611,PK,66.4512,30.9236,20.38,18.89,20.38,20.38,1011,16,10000,3.49,318,1647653985,1647697463,0
540,Washington,5815135,US,-120.5015,47.5001,2.79,2.79,0.52,4.40,1010,79,10000,1.31,301,1647698804,1647742362,100
541,Fort Nelson,5955902,CA,-122.7002,58.8053,-8.23,-8.23,-8.23,-8.23,1003,79,10000,1.03,360,1647699301,1647742921,75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# House the city coordinates in a variable.

coordinates = WeatherPy_df[["Latitude", "Longitude"]]

# House the humidity  values in a variable.

humidity = WeatherPy_df["Humidity"]

In [4]:
# Create Heatmap

# Set the output image size:

figure_layout = {'width': '900px','height': '600px','border': '1px solid black','padding': '1px','margin': '0 auto 0 auto'}

fig = gmaps.figure(layout = figure_layout, zoom_level=2,center = (15,25))

# Create heat layer

heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 3)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Convert the units in the ideal weather conditions to METRIC:

# A max temperature lower than 26.67 degrees Celcius (80 degrees F), but higher than 21.1 degrees Celcius (70 grees F).
# The max wind speed is 4.4704 m/s (10 miles per hour).

# Filter by max ideal temperature:

filter1_df = WeatherPy_df.loc[WeatherPy_df["Temp"] < 26.67, :]

# Filter by min ideal temperature:

filter2_df = filter1_df.loc[filter1_df["Temp"] > 21.1, :]

# Filter by max wind speed:

filter3_df = filter2_df.loc[filter2_df["Wind Speed"] < 4.4704, :]

# Filter by max wind speed:

filter4_df = filter3_df.loc[filter3_df["Cloudiness"] == 0, :]
ideal_weather_df = filter4_df.dropna()
ideal_weather_df

,City Name,City ID,Country Code,Longitude,Latitude,Temp,Feels Like Temp,Min Temp,Max Temp,Pressure,Humidity,Visibility,Wind Speed,Wind Temp,Sunrise,Sunset,Cloudiness
180,Puerto Escondido,3520994,MX,-97.0667,15.8500,23.46,24.16,23.46,23.46,1014,88,10000,3.60,20,1647693185,1647736735,0
242,Cam Ranh,1586350,VN,109.1591,11.9214,25.98,25.98,25.98,25.98,1009,89,10000,1.03,0,1647643708,1647687245,0
243,Ocós,3592086,GT,-92.1933,14.5094,26.10,26.10,26.10,26.10,1012,73,10000,3.33,322,1647692014,1647735567,0
467,Makakilo City,5850554,US,-158.0858,21.3469,21.53,21.72,20.90,22.78,1019,76,10000,3.60,50,1647707825,1647751378,0
471,Nānākuli,5851349,US,-158.1547,21.3906,22.19,22.37,21.69,23.58,1019,73,10000,2.57,30,1647707842,1647751394,0
527,Rāzole,1258269,IN,81.8333,16.4833,24.67,25.44,24.67,24.67,1005,86,10000,2.25,153,1647650273,1647693794,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=ideal_weather_df
hotel_df["Hotel Name"] = ""

In [7]:
len(hotel_df)

6

In [8]:
# Create a list to house the hotel names generated by the request.

Hotel_Names=[]  

# Store cities and their latitudes & logitudes (from hotel_df) as variables.

city = hotel_df["City Name"].tolist()
latitude = hotel_df['Latitude'].tolist()
longitude = hotel_df['Longitude'].tolist()


# Build URL and make Google Places API request.

for i in range(len(hotel_df)):
    coordinates = f"{latitude[i]}, {longitude[i]}"

    # Specify the parameters.
    
    params = {"location": coordinates, "types": "hotel", "keyword": "lodging", "radius": 5000, "key": g_key}
    
    # Specify the url that will be used as the basis of the request.
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:
        
        # Run the request and store the name of the hotel that is first returned.
        
        response = requests.get(url, params=params).json()
        Hotel_Names.append(response['results'][0]['name'])
    
    except:
        
        # If no data exists then add the following string to the hotel names list.
        
        Hotel_Names.append('NO HOTEL FOUND')
        
# Once the iterations are finished, add the hotel name list as a column in hotels_df.

hotel_df['Hotel Name'] = Hotel_Names 
hotel_df[["City Name", "Longitude", "Latitude", "Hotel Name", "Temp", "Cloudiness", "Wind Speed"]]

,City Name,Longitude,Latitude,Hotel Name,Temp,Cloudiness,Wind Speed
180,Puerto Escondido,-97.0667,15.8500,Selina Puerto Escondido,23.46,0,3.60
242,Cam Ranh,109.1591,11.9214,Kitty Hotel Cam Ranh,25.98,0,1.03
243,Ocós,-92.1933,14.5094,NO HOTEL FOUND,26.10,0,3.33
467,Makakilo City,-158.0858,21.3469,Barber's Point and ITT and Beach Cottages,21.53,0,3.60
471,Nānākuli,-158.1547,21.3906,NO HOTEL FOUND,22.19,0,2.57
527,Rāzole,81.8333,16.4833,Tarani In Hotel,24.67,0,2.25


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…